# get measured dot position in PFI mm coordinates

In [1]:
%matplotlib notebook

import os
import sys
import pandas as pd
import sep
import cv2
from copy import deepcopy

import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import math

In [2]:
im_data_dir = '/work/PFS/pfi/data/ait_subaru_202107/dot_measurements/2021-07-10'
repo_dir = '/work/PFS/repo/pfs_instdata'
ff_data_dir = '/work/PFS/pfi/data/ffdata'

## read data

### raw image data

In [3]:
filename=os.path.join(im_data_dir, 'PFSC22589400.fits')
with fits.open(filename) as hdul:
    data_dots=hdul[1].data

### measured dot positions (in 71M pix)

In [4]:
filename = os.path.join(repo_dir, 'data/pfi/dot', 'dot_measurements_subaru_20210710_el90_rot+00_ave.csv')
df = pd.read_csv(filename)
dot_71m_id = df['id']
dot_71m_x = df['x_dot']
dot_71m_y = df['y_dot']
dot_71m_r = df['r_dot']
df[:3]

,id,x_dot,y_dot,r_dot
0,0.0,5144.74,3509.03,10.0297
1,1.0,5238.65,3560.32,10.0301
2,2.0,5332.56,3611.62,10.0301


### designed dot position (in designed coordinate mm)

In [5]:
filename = os.path.join(repo_dir, 'data/pfi/dot', 'dot_design_asrd_20210422.txt')
dot_design_id, dot_design_x, dot_design_y = np.genfromtxt(filename, unpack=True, usecols=(0,1,2))

### measured ff positions (in 71M pix)

In [6]:
filename = os.path.join(repo_dir, 'data/pfi/dot', 'ff_subaru_20210710_el90_rot+00_ave.csv')
df = pd.read_csv(filename)
ff_71m_id = df['idx']
ff_71m_x = df['x_pixel']
ff_71m_y = df['y_pixel']
df[:3]

,idx,x_pixel,y_pixel
0,1,4848.882770,3672.585972
1,2,4653.825469,3771.350941
2,3,4482.099159,3878.708878


### ff positions (in PFI mm, T=20 degC)

In [7]:
filename = os.path.join(ff_data_dir, 'fiber_status_summary_FF_20C90DEG.txt')
ff_pfi_id, ff_pfi_x, ff_pfi_y = np.genfromtxt(filename, unpack=True, usecols=(1, 3, 4))

### current dot positions (in PFI mm)

In [8]:
filename = os.path.join(repo_dir, 'data/pfi/dot', 'black_dots_mm.csv')
df = pd.read_csv(filename)
dot_pfi_current_id = df['spotId']
dot_pfi_current_x = df['x']
dot_pfi_current_y = df['y']
dot_pfi_current_r = df['r']
df[:3]

,spotId,x,y,r
0,1,-5.353405,7.447845,0.750886
1,2,-12.345163,3.536719,0.750811
2,3,-5.462555,-0.562960,0.750919


### current ff position (in PFI mm)

In [9]:
filename = os.path.join(repo_dir, 'data/pfi', 'fiducial_positions.csv')
df = pd.read_csv(filename, comment='#')
ff_pfi_current_id = df['fiducialId']
ff_pfi_current_x = df['x_mm']
ff_pfi_current_y = df['y_mm']
df[:3]

,fiducialId,x_mm,y_mm
0,1,-13.838,11.133
1,2,-28.527,18.399
2,3,-41.500,26.353


## get conversion between 71M pix and ASRD designed mm

In [10]:
pts0=np.zeros((1,len(dot_71m_id),3))
pts0[0,:,0]=dot_71m_x
pts0[0,:,1]=dot_71m_y
pts0[0,:,2]=[0 for x in dot_71m_x]
pts1=np.zeros((1,len(dot_71m_id),3))
pts1[0,:,0]=dot_71m_x
pts1[0,:,1]=dot_71m_y
pts1[0,:,2]=[0 for x in dot_71m_x]
pts2=np.zeros((1,len(dot_design_id),2))
pts2[0,:,0]=dot_design_x
pts2[0,:,1]=dot_design_y

pts1 = np.float32(pts1)
pts2 = np.float32(pts2)

In [11]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(pts1, pts2, (250, 250), None, None)

In [12]:
## calculate transformed dot position ##
for i in range(len(pts0)):
    imgpoints2, _ = cv2.projectPoints(pts0[i].astype(np.float32),rvecs[i], tvecs[i], mtx, dist)
    
## calculate dot radius ##
pts0d = deepcopy(pts0)
pts0d[0,:,0] = pts0d[0,:,0] + 1.5

for i in range(len(pts0)):
    tmp1, _ = cv2.projectPoints(pts0[i].astype(np.float32),rvecs[i], tvecs[i], mtx, dist)
for i in range(len(pts0d)):
    tmp2, _ = cv2.projectPoints(pts0d[i].astype(np.float32),rvecs[i], tvecs[i], mtx, dist)
dot_rad = (tmp2[:,0,0] - tmp1[:,0,0]) / 2.0

dot_pos_trans = np.float32([[x,y,r] for x,y,r in zip(imgpoints2[:,0,0], imgpoints2[:,0,1], dot_rad[:])])

In [13]:
dot_design_trans_x = dot_pos_trans[:,0]
dot_design_trans_y = dot_pos_trans[:,1]
dot_design_trans_r = dot_pos_trans[:,2]

In [14]:
## check difference
fig, axe = plt.subplots(figsize=(7,6))
axe.set_xlabel(r'Designed (X or Y) position (mm)')
axe.set_ylabel(r'Difference (micron)')
axe.set_title(r'Difference between designed and transformed position')
axe.set_ylim(-25, 25)
axe.scatter(dot_design_x, 1000*(dot_design_trans_x - dot_design_x) , s=20, label='X')
axe.scatter(dot_design_y, 1000*(dot_design_trans_y - dot_design_y) , s=20, label='Y')
axe.legend(loc='upper left')

<IPython.core.display.Javascript object>

In [15]:
fig, axe = plt.subplots(figsize=(7,6))
axe.set_aspect('equal')
axe.set_xlabel(r'X (mm)')
axe.set_ylabel(r'Y (mm)')
axe.set_title(r'Difference between designed and transformed position')
axe.scatter(dot_design_x, dot_design_y, s=10)
axe.scatter(dot_design_trans_x, dot_design_trans_y, s=5)

<IPython.core.display.Javascript object>

## get ff position (in ASRD designed coordinate)

In [16]:
pts0=np.zeros((1,len(ff_71m_id),3))
pts0[0,:,0]=ff_71m_x
pts0[0,:,1]=ff_71m_y
pts0[0,:,2]=[0 for x in ff_71m_x]

In [17]:
for i in range(len(pts0)):
    imgpoints2, _ = cv2.projectPoints(pts0[i].astype(np.float32),rvecs[i], tvecs[i], mtx, dist)

In [18]:
ff_design_id = ff_71m_id.copy()
ff_design_x = imgpoints2[:,0,0]
ff_design_y = imgpoints2[:,0,1]

In [19]:
print(len(ff_design_id))
print(len(ff_pfi_id))

93
93


In [20]:
fig, axe = plt.subplots(figsize=(7,6))
axe.set_aspect('equal')
axe.set_xlabel(r'X (mm)')
axe.set_ylabel(r'Y (mm)')
axe.set_title(r'')
axe.scatter(dot_design_x, dot_design_y, s=10)
axe.scatter(ff_design_x, ff_design_y, s=5)

<IPython.core.display.Javascript object>

In [21]:
fig, axe = plt.subplots(figsize=(7,6))
axe.set_aspect('equal')
axe.set_xlabel(r'X (pix)')
axe.set_ylabel(r'Y (pix)')
axe.set_title(r'')
axe.scatter(dot_71m_x, dot_71m_y, s=10)
axe.scatter(ff_71m_x, ff_71m_y, s=5)

<IPython.core.display.Javascript object>

## get conversion between designed coordinate and PFI coordinate

In [22]:
fig, axe = plt.subplots(figsize=(7,6))
axe.set_aspect('equal')
axe.set_xlabel(r'X (mm)')
axe.set_ylabel(r'Y (mm)')
axe.set_title(r'')
axe.scatter(ff_design_x, ff_design_y, s=10)
axe.scatter(-1*ff_pfi_x, ff_pfi_y, s=5)
axe.scatter(ff_pfi_current_x, ff_pfi_current_y, s=3)

<IPython.core.display.Javascript object>

In [23]:
## check difference
fig, axe = plt.subplots(figsize=(7,6))
axe.set_aspect('equal')
axe.set_xlabel(r'$\Delta$X (mm)')
axe.set_ylabel(r'$\Delta$Y (mm)')
axe.set_title(r'Difference between designed and PFI position')
#axe.set_ylim(-25, 25)
axe.scatter((-ff_pfi_x - ff_design_x),(ff_pfi_y - ff_design_y) , s=20)

<IPython.core.display.Javascript object>

In [24]:
print(cv2.estimateAffine2D.__doc__)

estimateAffine2D(from, to[, inliers[, method[, ransacReprojThreshold[, maxIters[, confidence[, refineIters]]]]]]) -> retval, inliers
.   @brief Computes an optimal affine transformation between two 2D point sets.
.   
.   It computes
.   \f[
.   \begin{bmatrix}
.   x\\
.   y\\
.   \end{bmatrix}
.   =
.   \begin{bmatrix}
.   a_{11} & a_{12}\\
.   a_{21} & a_{22}\\
.   \end{bmatrix}
.   \begin{bmatrix}
.   X\\
.   Y\\
.   \end{bmatrix}
.   +
.   \begin{bmatrix}
.   b_1\\
.   b_2\\
.   \end{bmatrix}
.   \f]
.   
.   @param from First input 2D point set containing \f$(X,Y)\f$.
.   @param to Second input 2D point set containing \f$(x,y)\f$.
.   @param inliers Output vector indicating which points are inliers (1-inlier, 0-outlier).
.   @param method Robust method used to compute transformation. The following methods are possible:
.   -   @ref RANSAC - RANSAC-based robust method
.   -   @ref LMEDS - Least-Median robust method
.   RANSAC is the default method.
.   @param ransacReprojThreshold 

In [25]:
pts1 = np.zeros((1, len(ff_design_id), 2))
pts2 = np.zeros((1, len(ff_pfi_id), 2))
pts1[0, :, 0] = ff_design_x
pts1[0, :, 1] = ff_design_y
pts2[0, :, 0] = -1.0*ff_pfi_x
pts2[0, :, 1] = ff_pfi_y

#inlier=np.array([1 for x in ff_design_id])
afCoeff, inlier = cv2.estimateAffine2D(pts1, pts2)
inlier = inlier.reshape(len(inlier))
#afCoeff, inlier = cv2.estimateAffine2D(pts1, pts2, inlier)

M = {}
M['affineCoeff'] = afCoeff
M['xTrans'] = afCoeff[0, 2]
M['yTrans'] = afCoeff[1, 2]
M['xScale'] = np.sqrt(afCoeff[0, 0]**2 + afCoeff[0, 1]**2)
M['yScale'] = np.sqrt(afCoeff[1, 0]**2 + afCoeff[1, 1]**2)
M['angle'] = np.arctan2(
    afCoeff[1, 0] / np.sqrt(afCoeff[0, 0]**2 + afCoeff[0, 1]**2),
    afCoeff[1, 1] / np.sqrt(afCoeff[1, 0]**2 + afCoeff[1, 1]**2))

In [26]:
inlier

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1], dtype=uint8)

In [27]:
pts=np.zeros((1,len(ff_design_id),2))
pts[0,:,0]=ff_design_x
pts[0,:,1]=ff_design_y
afCor=cv2.transform(pts, M['affineCoeff'])
ff_design_trans_x=afCor[0,:,0]
ff_design_trans_y=afCor[0,:,1]

In [28]:
## check difference
fig, axe = plt.subplots(figsize=(7,6))
axe.set_xlabel(r'PFI (X or Y) position (mm)')
axe.set_ylabel(r'Difference (micron)')
axe.set_title(r'Difference between PFI and transformed position')
axe.set_ylim(-100, +100)
axe.scatter(-ff_pfi_x, 1000*(-1.0*ff_pfi_x - ff_design_trans_x) , s=20, label='X')
axe.scatter(ff_pfi_y, 1000*(ff_pfi_y - ff_design_trans_y) , s=20, label='Y')
axe.legend(loc='lower right')

<IPython.core.display.Javascript object>

## convert dot position from 71M pix to PFI mm

In [29]:
pts=np.zeros((1,len(dot_design_id),2))
pts[0,:,0]=dot_design_x
pts[0,:,1]=dot_design_y
afCor=cv2.transform(pts, M['affineCoeff'])
dot_pfi_x=afCor[0,:,0]
dot_pfi_y=afCor[0,:,1]

In [30]:
dot_radius = 1.5 / 2.0
pts1=np.zeros((1,len(dot_design_id),2))
pts1[0,:,0]=dot_design_x-dot_radius
pts1[0,:,1]=dot_design_y
afCor1=cv2.transform(pts1, M['affineCoeff'])
pts2=np.zeros((1,len(dot_design_id),2))
pts2[0,:,0]=dot_design_x+dot_radius
pts2[0,:,1]=dot_design_y
afCor2=cv2.transform(pts2, M['affineCoeff'])
dot_pfi_r=(afCor2[0,:,0] - afCor1[0,:,0]) / 2.0

In [31]:
## check distribution
fig, axe = plt.subplots(figsize=(7,6))
axe.set_aspect('equal')
axe.set_xlabel(r'X (mm)')
axe.set_ylabel(r'Y (mm)')
axe.set_title(r'')
axe.scatter(dot_pfi_x, dot_pfi_y, s=10)
axe.scatter(dot_pfi_current_x, dot_pfi_current_y, s=5)

<IPython.core.display.Javascript object>

In [ ]:
## cross-match datapoints
diff_pfi_x = []
diff_pfi_y = []
for x1,y1 in zip(dot_pfi_x, dot_pfi_y):
    for x2,y2 in zip(dot_pfi_current_x, dot_pfi_current_y):
        d2 = (x1 - x2)**2 + (y1 - y2)**2
        if d2 <= 1.0**2:
            diff_pfi_x.append(x1 - x2)
            diff_pfi_y.append(y1 - y2)
diff_pfi_x = np.array(diff_pfi_x)
diff_pfi_y = np.array(diff_pfi_y)
print(len(diff_pfi_x))
print(len(diff_pfi_y))

In [ ]:
## check difference
fig, axe = plt.subplots(figsize=(7,6))
axe.set_xlabel(r'X or Y position (mm)')
axe.set_ylabel(r'Difference (micron)')
axe.set_title(r'Difference between designed and transformed position')
axe.scatter(dot_pfi_x, 1000*diff_pfi_x , s=20, label='X')
axe.scatter(dot_pfi_y, 1000*diff_pfi_y , s=20, label='Y')
axe.legend(loc='upper left')

In [ ]:
fig, axe = plt.subplots(figsize=(7,6))
axe.set_aspect('equal')
axe.set_xlabel(r'X (mm)')
axe.set_ylabel(r'Y (mm)')
axe.set_title(r'')
axe.scatter(-ff_pfi_x, ff_pfi_y, s=5)
axe.scatter(ff_pfi_current_x, ff_pfi_current_y, s=3)

## writeto

In [ ]:
df = pd.DataFrame({
    'dotId': np.array(dot_design_id, dtype='i4'),
    'x': np.array(dot_pfi_x, dtype='f4'),
    'y': np.array(dot_pfi_y, dtype='f4'),
    'r': np.array(dot_pfi_r, dtype='f4')
})
df[:3]

In [ ]:
df.to_csv('black_dots_mm_kiyoyabe.csv', index=False)

## sanity check

In [ ]:
filename = os.path.join(repo_dir, 'data/pfi/dot', 'black_dots_mm.csv')
df = pd.read_csv(filename)
dot_id1 = df['spotId']
dot_x1 = df['x']
dot_y1 = df['y']
dot_r1 = df['r']

In [ ]:
filename = os.path.join('black_dots_mm_kiyoyabe.csv')
df = pd.read_csv(filename)
dot_id2 = df['dotId']
dot_x2 = df['x']
dot_y2 = df['y']
dot_r2 = df['r']

In [ ]:
fig, axe = plt.subplots(figsize=(7,6))
axe.set_aspect('equal')
axe.set_xlabel(r'X (mm)')
axe.set_ylabel(r'Y (mm)')
axe.set_title(r'')
axe.scatter(dot_x1, dot_y1, s=10)
axe.scatter(dot_x2, dot_y2, s=5)